#### Pretrain ####
This notebook was used to pretrain the model.

In [ ]:
# some necessary imports and setting seed

import torch
import torchvision
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
from pytorch_revgrad import RevGrad
import random
import torch.nn as nn
import torch.nn.functional as F
import model
import tt_simnet as sim

torch.manual_seed(0)

In [2]:
# loading in (and if necessary downloading) the MNIST and USPS datasets

transformation = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.RandomCrop((28, 28)),
    transforms.ToTensor()
])

train_MNIST = datasets.MNIST('', train=True, download=True, transform=transformation)
test_MNIST = datasets.MNIST('', train=False, download=True, transform=transformation)

train_USPS = datasets.USPS('USPS', train=True, download=True, transform=transformation)
test_USPS = datasets.USPS('USPS', train=False, download=True, transform=transformation)

trainset_MNIST = torch.utils.data.DataLoader(train_MNIST, batch_size=32, shuffle=True)
testset_MNIST = torch.utils.data.DataLoader(test_MNIST, batch_size=32, shuffle=False)

trainset_USPS = torch.utils.data.DataLoader(train_USPS, batch_size=32, shuffle=True)
testset_USPS = torch.utils.data.DataLoader(test_USPS, batch_size=32, shuffle=False)

In [3]:
# if available setting the device to GPU

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
# setting up pretrain parameters

net = model.PretrainNet().to(device)
optimizer_net = torch.optim.Adam(net.parameters(), lr=1e-3)
num_epochs = 10
save = False
reg = False
gamma_reg = 0.01

In [1]:
net = sim.pretrain(device, net, optimizer_net, num_epochs, trainset_MNIST, regularization=reg, gamma_reg=gamma_reg)

In [ ]:
if save == True:
    if reg == True:
        PATH = "standard_encoder_reg.pt"
    elif reg == False:
        PATH = "standard_encoder_noreg.pt"
        
    torch.save(net.encoder.state_dict(), PATH)